In [ ]:
from langchain_community.document_loaders import CSVLoader
from langchain.chat_models import init_chat_model
from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings
import os
import numpy as np
from pinecone import Pinecone as PineconeClient
from langchain.docstore.document import Document
import pandas as pd
from uuid import uuid4
from dotenv import load_dotenv
# Load biến từ .env
load_dotenv()

### 1. Data Analysis

In [4]:
data = pd.read_csv("hotels_users_ratings.csv")
data.head(10)

,URL Hotel,Location,HotelID,Name Hotel,Descriptions,Address,UserID,User,Rating
0,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1187,Thảo,6
1,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1284,Tran,5
2,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",5866,Tho,6
3,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",3033,Tuan,9
4,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1406,Phan,7
5,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",841,Hong,8
6,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",335,Thanh,6
7,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",3170,Vu,7
8,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",4446,Trung,8
9,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",5368,Hai,8


In [5]:
data['Location'].value_counts()

,count
Location,
Hà Nội,5695
TP. Hồ Chí Minh,5421
Đà Nẵng,5337
Đà Lạt,5275
Hội An,5130
Phú Quốc,2915
Nha Trang,2845
Vũng Tàu,2716
Sa Pa,1761


In [6]:
np.unique(data['Location']), np.unique(data['Rating'])

(array(['Huế', 'Hà Nội', 'Hội An', 'Nha Trang', 'Phú Quốc', 'Sa Pa',
        'TP. Hồ Chí Minh', 'Vũng Tàu', 'Đà Lạt', 'Đà Nẵng'], dtype=object),
 array([ 4,  5,  6,  7,  8,  9, 10]))

In [7]:
# Kiểm tra dữ liệu trùng nhau
results = data.loc[data['Name Hotel'].isin(["HANZ Binh Duong Hotel Dalat"])]
results

,URL Hotel,Location,HotelID,Name Hotel,Descriptions,Address,UserID,User,Rating
406,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",1049,Ngoan,5
407,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",1932,Trần,6
408,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",5389,Hieu,6
409,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",4762,Thuỳ,6
410,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",3476,Duy,8
411,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",3033,Tuan,7
412,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",2479,Trâm,5
413,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",5787,Béo,5
414,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",1337,Thân,8
415,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",4252,Nguyễn,5


In [8]:
data['Max_Rating'] = data.groupby(['Name Hotel', 'UserID'])['Rating'].transform('max')
# Lọc các hàng có Rating bằng giá trị lớn nhất
new_data = data[data['Rating'] == data['Max_Rating']]

# Xóa cột tạm thời
new_data = new_data.drop(columns=['Max_Rating'])
# Xóa hàng trùng
new_data = new_data.drop_duplicates(subset=['HotelID', 'UserID', 'Rating'])

new_data.head(10)

,URL Hotel,Location,HotelID,Name Hotel,Descriptions,Address,UserID,User,Rating
0,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1187,Thảo,6
1,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1284,Tran,5
2,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",5866,Tho,6
3,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",3033,Tuan,9
4,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",1406,Phan,7
5,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",841,Hong,8
6,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",335,Thanh,6
7,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",3170,Vu,7
8,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",4446,Trung,8
9,https://www.booking.com/hotel/vn/dalat-wind.vi...,Đà Lạt,4064,Dalat Wind Deluxe Hotel,"Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...",5368,Hai,8


In [9]:
# Dữ liệu sau khi loại bỏ một số hàng trùng
new_data.loc[new_data['Name Hotel'].isin(["HANZ Binh Duong Hotel Dalat"])]

,URL Hotel,Location,HotelID,Name Hotel,Descriptions,Address,UserID,User,Rating
408,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",5389,Hieu,6
409,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",4762,Thuỳ,6
410,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",3476,Duy,8
411,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",3033,Tuan,7
413,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",5787,Béo,5
414,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",1337,Thân,8
769,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",1049,Ngoan,7
770,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",1932,Trần,8
775,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",2479,Trâm,9
778,https://www.booking.com/hotel/vn/binh-duong-da...,Đà Lạt,1873,HANZ Binh Duong Hotel Dalat,"Nằm ở thành phố Đà Lạt, cách Vườn hoa Đà Lạt 2...","34/1 Hung Vuong, Ward 9, Đà Lạt, Việt Nam",4252,Nguyễn,7


In [10]:
print(f"The length of original data: {len(data)}")
print(f"The length of processed data: {len(new_data)}")

The length of original data: 38801
The length of processed data: 32963


### 2. Create metadata and document

In [13]:
def round_to_half(number):
    """Làm tròn số đến 0.5 gần nhất theo quy tắc:
    - < 0.25 làm tròn xuống .0
    - >= 0.25 và < 0.75 làm tròn đến .5
    - >= 0.75 làm tròn lên .0 tiếp theo
    """
    base = int(number)  # Phần nguyên
    decimal = number - base  # Phần thập phân

    if decimal < 0.25:
        return float(base)
    elif decimal < 0.75:
        return base + 0.5
    else:
        return float(base + 1)

In [15]:
def create_document_metadata(df):
  documents = []
  current_id = set()
  for idx, row in df.iterrows():
    # Loại bỏ các khách sạn trùng với nhiều userid
    if row['HotelID'] not in current_id:
        content = f"Tên khách sạn: {row['Name Hotel']}\n"
        content += f"Mô tả: {row['Descriptions']}\n"
        content += f"Địa chỉ: {row['Address']}\n"
        content += f"URL khách sạn: {row['URL Hotel']}\n"

        ratings = list(set(new_data.loc[new_data['HotelID'] == row['HotelID'], 'Rating']))
        avg_rating = sum(ratings) / len(ratings)
        rounded_rating = round_to_half(avg_rating)

        metadata = {
            'hotel_id': row['HotelID'],
            'hotel_name': row['Name Hotel'],
            'rating': rounded_rating,
            'location': row['Location']
        }
        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)

    current_id.add(row['HotelID'])

  return documents

In [16]:
documents = create_document_metadata(new_data)
documents[0:2]

[Document(metadata={'hotel_id': 4064, 'hotel_name': 'Dalat Wind Deluxe Hotel', 'rating': 7.0, 'location': 'Đà Lạt'}, page_content='Tên khách sạn: Dalat Wind Deluxe Hotel\nMô tả: Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hương 500 m, Dalat Wind Deluxe Hotel là khách sạn 2 sao có lễ tân 24 giờ, sảnh khách chung, Wi-Fi và chỗ đỗ xe riêng miễn phí. Tại khách sạn, tất cả các phòng đều có bàn làm việc, TV màn hình phẳng, ấm đun nước và phòng tắm riêng với chậu rửa vệ sinh (bidet). Một số phòng còn có ban công. Khách sạn phục vụ bữa sáng gọi món hàng ngày. Du khách có thể dùng bữa tại nhà hàng trong khuôn viên, nơi chuyên phục vụ các món nướng/BBQ. Du khách cũng có thể thư giãn trên sân hiên tắm nắng. Các điểm tham quan nổi tiếng gần Dalat Wind Deluxe Hotel bao gồm Vườn hoa Đà Lạt, Quảng trường Lâm Viên và Công viên Yersin Đà Lạt. Sân bay gần nhất là sân bay Liên Khương, cách chỗ nghỉ 23 km.\nĐịa chỉ: Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Việt Nam\nURL khách sạn: https://www.booking.co

### 3. Chunk data

In [17]:
def data_chunk(docs, chunk_size, chunk_overlap):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap,
      length_function=len
  )
  chunk = text_splitter.split_documents(docs)
  return chunk

In [40]:
chunks = data_chunk(docs=documents, chunk_size=1000, chunk_overlap=100)
print(f"Chunk length: {len(chunks)}")
print("Chunk: 1")
print(f"Content: {chunks[0].page_content}")
print(f"Metadata: {chunks[0].metadata}")

Chunk length: 10159
Chunk: 1
Content: Tên khách sạn: Dalat Wind Deluxe Hotel
Mô tả: Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hương 500 m, Dalat Wind Deluxe Hotel là khách sạn 2 sao có lễ tân 24 giờ, sảnh khách chung, Wi-Fi và chỗ đỗ xe riêng miễn phí. Tại khách sạn, tất cả các phòng đều có bàn làm việc, TV màn hình phẳng, ấm đun nước và phòng tắm riêng với chậu rửa vệ sinh (bidet). Một số phòng còn có ban công. Khách sạn phục vụ bữa sáng gọi món hàng ngày. Du khách có thể dùng bữa tại nhà hàng trong khuôn viên, nơi chuyên phục vụ các món nướng/BBQ. Du khách cũng có thể thư giãn trên sân hiên tắm nắng. Các điểm tham quan nổi tiếng gần Dalat Wind Deluxe Hotel bao gồm Vườn hoa Đà Lạt, Quảng trường Lâm Viên và Công viên Yersin Đà Lạt. Sân bay gần nhất là sân bay Liên Khương, cách chỗ nghỉ 23 km.
Địa chỉ: Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Việt Nam
Metadata: {'hotel_id': 4064, 'hotel_name': 'Dalat Wind Deluxe Hotel', 'rating': 7.0, 'location': 'Đà Lạt'}


### 4. Putting embedding into vector database Pinecone

In [19]:
embedding = GPT4AllEmbeddings()
print(f"Dimension of embedding: {np.array(embedding.embed_query('How are you')).shape}")

Downloading: 100%|██████████| 45.9M/45.9M [00:00<00:00, 63.6MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 480MiB/s]


Dimension of embedding: (384,)


In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = PineconeClient(
      api_key=PINECONE_API_KEY,
  )
index_name = "langchainretrieval"
index = pc.Index(index_name)

In [21]:
vector_store = PineconeVectorStore.from_documents(
    chunks, embedding, index_name=index_name
)

In [28]:
results = vector_store.similarity_search(
    "Khách sạn ở Đà Lạt với rating là 8",
    k=2,
    filter={"location": "Đà Lạt"}
)

In [29]:
for res in results:
  print(f"Response: \n{res.page_content}")

Response: 
Tên khách sạn: DALAT STREAM HOTEL-Khách sạn đẹp Đà Lạt
Response: 
Tên khách sạn: VILLADECELIA
Mô tả: Biệt thự cung cấp miễn phí cả WiFi lẫn chỗ đỗ xe riêng. Mỗi căn tại đây đều có sân trong, khu vực bếp ăn đầy đủ tiện nghi với lò vi sóng, khu vực ghế ngồi, TV màn hình phẳng, máy giặt cũng như phòng tắm riêng đi kèm vòi xịt/chậu rửa vệ sinh và dép đi trong phòng. Các căn còn được trang bị tủ lạnh, lò nướng, bếp nấu ăn và ấm đun nước. VILLADECELIA nằm cách CLB chơi golf Đà Lạt Palace 1,6 km và Vườn hoa Đà Lạt 2,2 km. Sân bay gần nhất là sân bay Liên Khương, cách chỗ nghỉ 31 km.
Địa chỉ: 98 Đường Thông Thiên Học, Đà Lạt, Việt Nam
